In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import requests

import pandas as pd
import os.path

import time

# 1 - Creating Functions for Further Use

## function 1: returns the html content of the page at the given link

In [32]:
def scrape_html_content(url):
    # Scraping logic to get HTML content
    try:
        driver.get(url)
        time.sleep(1)
        try:
            # Find all "Show more data" buttons
            show_more_buttons = driver.find_elements(By.XPATH, '//*[@id=":0"]/div/div[2]/div[1]/div[4]/div[3]/div/div/div/div/div/div[2]/div')

            # Click each "Show more data" button
            for button in show_more_buttons:
                button.click()

                # Wait for the page to load the additional data
                wait = WebDriverWait(driver, 5)
                wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id=":0"]/div/div[2]/div[1]/div[4]/div[3]/div/div/div/div/div/div[2]/div')))

        except NoSuchElementException:
            pass

        # Retrieve the HTML content of the entire page
        html_content = driver.page_source
        return html_content
    
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None
    # Close the driver
    driver.quit()

## Function 2: returns price data to an empty created DataFrame

In [33]:
def get_df_prices_from_url(url):
    # Parse the HTML content using BeautifulSoup
    html_content = scrape_html_content(url)
    if not html_content:
        return pd.DataFrame()  # Return an empty DataFrame if no HTML content

    soup = BeautifulSoup(html_content, "lxml")

    # Find all tables with prices in html content
    tables_tags = soup.find_all('div', class_='zen-roomspagerooms-room')
    if not tables_tags:
        return pd.DataFrame()  # Return an empty DataFrame if no tables found

    # Create an empty list to store the data
    data = []

    # 'tables_tags' is the list of bs4.element.Tag objects containing tables
    for table in tables_tags:
        # Extract data from each row in the table
        rows = table.find_all('tr')
        # Table name is the name of the room
        table_name = table.find('div', class_='zenroomspagerate-name-title')
        if table_name:
            table_name = table_name.text.strip()
        # Description of facilities of each room
        Descriptive_elements = table.select('[class^="zenroomspageroom-header-content-amenity"]')
        # Make it a list
        Descriptive_elements_texts = [element.text.strip() for element in Descriptive_elements]
        for row in rows:
            # Create a new temporary list to store the data for this row
            row_data = [url, table_name, Descriptive_elements_texts]
            # Extract data from each cell in the row
            cells = row.find_all('div', class_=['valueadds-item-title-inner', 'zenroomspage-b2c-rates-price-amount'])
            for i, cell in enumerate(cells):
                # Process each cell as needed and append its content to the row_data list
                row_data.append(cell.text.strip())
                if i == 0:
                    row_data.append(str(cell))
            # Append the row_data list to the data list
            data.append(row_data)

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    return df

# 2 - Collecting Information about Prices from All Links

In [38]:
# Create an empty list to store the prices data
data_prices_total = []
# Create a DataFrame from the data_prices list
df_prices_total = pd.DataFrame(data_prices_total)

In [ ]:
# Read the Excel file
df_urls = pd.read_excel('URLs Murm test.xlsx')

# Extract URLs from a specific column
urls = df_urls['URL'].tolist()


options = Options()
options.headless = True
options.add_experimental_option('prefs', {
    'profile.managed_default_content_settings.images': 2,
#    'profile.managed_default_content_settings.mixed_script': 2,
    'profile.managed_default_content_settings.media_stream': 2,
    'profile.managed_default_content_settings.stylesheets': 2,
        })

driver = webdriver.Chrome(options=options)

for url in urls:
    df_prices_temp = get_df_prices_from_url(url)
    df_prices_total = pd.concat([df_prices_total, df_prices_temp])

In [25]:
df_prices_total

,0,1,2,3,4,5,6,7,8,9,10
0,https://ostrovok.ru/hotel/russia/murmansk/mid9...,Двухместные апартаменты Standard с балконом 2 ...,"[17 м2, Собственная ванная комната, Кухня, Выс...",2×,"<div class=""valueadds-item-title-inner valuead...",Питание не включено,Для некурящих,Бесплатная отмена до 17 марта*,Оплата сейчас,157 €,NaN
1,https://ostrovok.ru/hotel/russia/murmansk/mid9...,Двухместные апартаменты Standard с балконом 2 ...,"[17 м2, Собственная ванная комната, Кухня, Выс...",2×,"<div class=""valueadds-item-title-inner valuead...",Питание не включено,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,158 €,NaN
0,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместный номер SMART Superior двуспальная к...,"[21 м2, Собственная ванная комната]",Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Питание не включено,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,508 €,NaN
1,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместный номер SMART Superior двуспальная к...,"[21 м2, Собственная ванная комната]",Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Завтрак включён,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,640 €,NaN
2,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместный номер SMART Superior deluxe двуспа...,[Собственная ванная комната],Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Питание не включено,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,568 €,NaN
3,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместный номер SMART Superior deluxe двуспа...,[Собственная ванная комната],Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Завтрак включён,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,699 €,NaN
4,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместный SMART полулюкс двуспальная кровать,"[Собственная ванная комната, Туалетные принадл...",Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Питание не включено,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,658 €,NaN
5,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместный SMART полулюкс двуспальная кровать,"[Собственная ванная комната, Туалетные принадл...",Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Завтрак включён,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,789 €,NaN
6,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместные апартаменты SMART двуспальная кровать,[Собственная ванная комната],Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Питание не включено,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,747 €,NaN
7,https://ostrovok.ru/hotel/russia/murmansk/mid6...,Двухместные апартаменты SMART двуспальная кровать,[Собственная ванная комната],Двуспальная кровать,"<div class=""valueadds-item-title-inner"">Двуспа...",Завтрак включён,Для некурящих,Бесплатная отмена до 17 марта*,Оплата на месте - Для бронирования нужна карта,879 €,NaN


In [7]:
df_prices_total.to_excel('murm test prices table.xlsx', index=False)